In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pandas as pd
from collections import namedtuple
from collections import UserList
from sklearn import tree

# new packages
from collections import defaultdict
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_linnerud
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, MinMaxScaler
import os
from sklearn.ensemble import RandomForestClassifier


In [3]:
## Initiaze logger
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')
logger = logging.getLogger('microsegmenter')

In [4]:
from decision_tree_more_features import *
from helper import preprocess, convert_onehot_v1
from load_config import load_yaml, dump_yaml, safe_load_yaml

In [5]:
# Load config files
general_config = safe_load_yaml('general_config.yaml')

In [6]:
# Load files
df = pd.read_csv(general_config['dataset']['data_path'])

In [7]:
ordinal_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder()
X, y, column_names, dtypes, target_names = preprocess(df, ordinal_encoder, onehot_encoder)

Your desired maximum number of one hot columns is not valid, setting to be the current maximum number of one hot columns.


In [8]:
rf_model = RandomForestClassifier(n_estimators = general_config['model']['random_forest']['n_estimators'],
                                  max_leaf_nodes = general_config['model']['random_forest']['max_leaf_nodes'],
                                  random_state = general_config['model']['random_forest']['random_state'])
rf_model.fit(X, y)

RandomForestClassifier(max_leaf_nodes=10, random_state=0)

In [9]:
# Remove duplicated decision paths and keep the decision path with the most narrowed condition if narrowed_condition = True
# Otherwise, no need to check duplications
def remove_duplicates(decision_paths, narrowed_condition = True):
    final_dic = {} # store each feature set with the corresponding threshold and index in the decision_paths
    for i, decision in enumerate(decision_paths):
        key_store = () # store the current decision feature set
        value_store = {} # store the mapping between feature and the corresponding threshold and index in the decision_paths
        operator_dic = {} # store the mapping between current feature and its corresponding operator
        for rule in decision:
            feature = rule.feature # current feature
            # keep updating the operator for each feature
            operator_dic[feature] = rule.operator # store the current operator in operator_dic
            key_store += (feature, ) # add the current feature to the feature set
            value_store[feature] = [rule.threshold, i] 
        key_store = tuple(sorted(key_store)) # sort the feature set to make sure the key remain the same for same feature set
        
        if narrowed_condition:
            if key_store in final_dic: # if the final dictionary contains the current feature set
                for tup in key_store: # loop through the key_store
                    # if any of the feature shows a more narrowed condition, replace the feature set with value in the final_dic
                    if operator_dic[tup] == "<=":
                        if value_store[tup][0] < final_dic[key_store][tup][0]:
                            final_dic[key_store] = value_store
                            break
                    else:
                        if value_store[tup][0] > final_dic[key_store][tup][0]:
                            final_dic[key_store] = value_store
                            break
            else: # if the current key_store is not in the final_dic, store it in the final_dic
                final_dic[key_store] = value_store
        else:
            final_dic[key_store] = value_store
    # record the new_paths
    new_paths = []
    for value in final_dic.values():
        subvalue = list(value.values())
        cur_index = subvalue[0][1]
        new_paths.append(decision_paths[cur_index])
        
    return new_paths

In [10]:
%%time
df_dict = {}
rule_outcome_map = {}
for i, (criteria_key, criteria_value) in enumerate(general_config['criteria'].items()):
    logger.info(f"Building the random forest for criteria {i}.")
    decision_paths = []
    for j, est in enumerate(rf_model.estimators_):
        interested_class = criteria_value['interested_class']
        min_samples = criteria_value['min_samples']
        lift = criteria_value['lift']
        base_rate = criteria_value['base_rate']
        tree = Track_DT(est, column_names, target_names, dtypes, interested_class,
                       min_samples, lift, base_rate, rule_outcome_map)
        cur_paths = tree.decision_paths
        for path in cur_paths:
            decision_paths.append(path)
        decision_paths = remove_duplicates(decision_paths)
        #decision_paths = list(set(decision_paths))
    df_dict[f'summary {i}'] = tree.summary(decision_paths)


2022-07-13 10:06:14,678 :: INFO :: Building the random forest for criteria 0.
2022-07-13 10:06:14,681 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.11  0.075 0.08  0.295 0.44 ] , 128
2022-07-13 10:06:14,681 :: INFO :: Distribution(%) of interested class in the node : HIGH_BP -  1 : [0.075]
2022-07-13 10:06:14,682 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,683 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.         0.         0.13333333 0.49166667 0.375     ] , 75
2022-07-13 10:06:14,683 :: INFO :: Distribution(%) of interested class in the node : NORMAL_BP -  1 : [0.]
2022-07-13 10:06:14,683 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,684 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.         0.         0.24615385 0.35384615 0.4       ] , 39
2022-07-13 10

2022-07-13 10:06:14,712 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.11  0.075 0.08  0.295 0.44 ] , 128
2022-07-13 10:06:14,712 :: INFO :: Distribution(%) of interested class in the node : HIGH_BP -  1 : [0.075]
2022-07-13 10:06:14,713 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,714 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.275  0.1875 0.     0.     0.5375] , 53
2022-07-13 10:06:14,714 :: INFO :: Distribution(%) of interested class in the node : Age -  1 : [0.1875]
2022-07-13 10:06:14,714 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,715 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.41509434 0.         0.         0.         0.58490566] , 35
2022-07-13 10:06:14,715 :: INFO :: Distribution(%) of interested class in the node : Na_to_K -  1 : [0.]
2022-07-

2022-07-13 10:06:14,747 :: INFO :: Appending Rule Na_to_K <= 14.837500095367432
2022-07-13 10:06:14,747 :: INFO :: List of rules added to decision path so far
2022-07-13 10:06:14,747 :: INFO :: Na_to_K <= 14.837500095367432 , node type : float64
2022-07-13 10:06:14,748 :: INFO :: Appending Rule LOW_BP <= 0.5
2022-07-13 10:06:14,748 :: INFO :: List of rules added to decision path so far
2022-07-13 10:06:14,749 :: INFO :: Na_to_K <= 14.837500095367432 , node type : float64
2022-07-13 10:06:14,749 :: INFO :: LOW_BP <= 0.5 , node type : object
2022-07-13 10:06:14,749 :: INFO :: Appending Rule NORMAL_BP <= 0.5
2022-07-13 10:06:14,750 :: INFO :: List of rules added to decision path so far
2022-07-13 10:06:14,750 :: INFO :: NORMAL_BP <= 0.5 , node type : object
2022-07-13 10:06:14,750 :: INFO :: Na_to_K <= 14.837500095367432 , node type : float64
2022-07-13 10:06:14,750 :: INFO :: LOW_BP <= 0.5 , node type : object
2022-07-13 10:06:14,751 :: INFO :: Distribution(%) of classes in the node [0. 

2022-07-13 10:06:14,778 :: INFO :: Distribution(%) of interested class in the node : LOW_BP -  1 : [0.12413793]
2022-07-13 10:06:14,778 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,779 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.13  0.09  0.09  0.225 0.465] , 127
2022-07-13 10:06:14,779 :: INFO :: Distribution(%) of interested class in the node : NORMAL_BP -  1 : [0.09]
2022-07-13 10:06:14,779 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,780 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.         0.         0.         0.58181818 0.41818182] , 36
2022-07-13 10:06:14,780 :: INFO :: Distribution(%) of interested class in the node : Age -  1 : [0.]
2022-07-13 10:06:14,781 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,781 :: INFO :: Distribution(%) of c

2022-07-13 10:06:14,806 :: INFO :: Distribution(%) of interested class in the node : Age -  1 : [0.]
2022-07-13 10:06:14,806 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,807 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.         0.         0.33333333 0.16666667 0.5       ] , 37
2022-07-13 10:06:14,807 :: INFO :: Distribution(%) of interested class in the node : NORMAL_Cholesterol -  1 : [0.]
2022-07-13 10:06:14,808 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,808 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.         0.         0.55555556 0.         0.44444444] , 20
2022-07-13 10:06:14,808 :: INFO :: Distribution(%) of interested class in the node : HIGH_Cholesterol -  1 : [0.]
2022-07-13 10:06:14,809 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:14,80

...

2022-07-13 10:06:39,563 :: INFO :: Distribution(%) of interested class in the node : Na_to_K -  3 : [0.285]
2022-07-13 10:06:39,563 :: INFO :: Evaluation of node to meet the microsegmentation criteria : True
2022-07-13 10:06:39,564 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.23148148 0.11111111 0.12962963 0.52777778 0.        ] , 67
2022-07-13 10:06:39,564 :: INFO :: Distribution(%) of interested class in the node : HIGH_Cholesterol -  3 : [0.52777778]
2022-07-13 10:06:39,564 :: INFO :: Evaluation of node to meet the microsegmentation criteria : True
2022-07-13 10:06:39,565 :: INFO :: Appending Rule Na_to_K <= 14.61549997329712
2022-07-13 10:06:39,565 :: INFO :: List of rules added to decision path so far
2022-07-13 10:06:39,566 :: INFO :: Na_to_K <= 14.61549997329712 , node type : float64
2022-07-13 10:06:39,566 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.16129032 0.14516129 0.         0.69354839

2022-07-13 10:06:39,590 :: INFO :: Distribution(%) of interested class in the node : HIGH_BP -  3 : [0.45833333]
2022-07-13 10:06:39,590 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,591 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [1. 0. 0. 0. 0.] , 2
2022-07-13 10:06:39,591 :: INFO :: Distribution(%) of interested class in the node : HIGH_Cholesterol -  3 : [0.]
2022-07-13 10:06:39,591 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,592 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.25       0.         0.29166667 0.45833333 0.        ] , 13
2022-07-13 10:06:39,592 :: INFO :: Distribution(%) of interested class in the node : HIGH_BP -  3 : [0.45833333]
2022-07-13 10:06:39,592 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,592 :: INFO :: Distribution(%) of 

current_dp:  [Rule(feature='Na_to_K', operator='<=', threshold=14.828500270843506, feature_type=dtype('float64'))]
Branch: left, Node type : inter , Feature : NORMAL_Cholesterol ,  Parent : HIGH_BP , Left Child : LOW_BP , Right Child : HIGH_Cholesterol , NodeID : 6
current_dp:  [Rule(feature='HIGH_BP', operator='<=', threshold=0.5, feature_type=dtype('O')), Rule(feature='Na_to_K', operator='<=', threshold=14.828500270843506, feature_type=dtype('float64'))]
Branch: left, Node type : inter , Feature : LOW_BP ,  Parent : NORMAL_Cholesterol , Left Child : HIGH_Cholesterol , Right Child : HIGH_Cholesterol , NodeID : 8
Branch: left, Node type : leaf , Feature : HIGH_Cholesterol , Parent : LOW_BP , NodeID : 10
Back Tracking!!
Branch: left, Node type : inter , Feature : LOW_BP ,  Parent : NORMAL_Cholesterol , Left Child : HIGH_Cholesterol , Right Child : HIGH_Cholesterol , NodeID : 8
Branch: right, Node type : leaf , Feature : HIGH_Cholesterol , Parent : LOW_BP , NodeID : 9
Back Tracking!!
Bra

2022-07-13 10:06:39,599 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,600 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0. 1. 0. 0. 0.] , 2
2022-07-13 10:06:39,600 :: INFO :: Distribution(%) of interested class in the node : HIGH_Cholesterol -  3 : [0.]
2022-07-13 10:06:39,600 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,601 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.75 0.25 0.   0.   0.  ] , 8
2022-07-13 10:06:39,601 :: INFO :: Distribution(%) of interested class in the node : Age -  3 : [0.]
2022-07-13 10:06:39,601 :: INFO :: Evaluation of node to meet the microsegmentation criteria : False
2022-07-13 10:06:39,602 :: INFO :: Distribution(%) of classes in the node [0. 1. 2. 3. 4.] and total samples: [0.40909091 0.13636364 0.31818182 0.13636364 0.        ] , 16
2022-07-13 10:06:39,602 :: INFO :: Dis

Branch: right, Node type : leaf , Feature : HIGH_Cholesterol , Parent : Age , NodeID : 6
Back Tracking!!
Branch: right, Node type : inter , Feature : Age ,  Parent : HIGH_BP , Left Child : HIGH_Cholesterol , Right Child : HIGH_Cholesterol , NodeID : 5
Back Tracking!!
Branch: right, Node type : inter , Feature : HIGH_BP ,  Parent : M_Sex , Left Child : HIGH_Cholesterol , Right Child : Age , NodeID : 4
Back Tracking!!
Branch: right, Node type : inter , Feature : M_Sex ,  Parent : HIGH_Cholesterol , Left Child : HIGH_BP , Right Child : HIGH_BP , NodeID : 3
Back Tracking!!
Branch: left, Node type : inter , Feature : HIGH_Cholesterol ,  Parent : Na_to_K , Left Child : HIGH_Cholesterol , Right Child : M_Sex , NodeID : 2
Back Tracking!!
current_dp:  [Rule(feature='Na_to_K', operator='<=', threshold=14.61549997329712, feature_type=dtype('float64'))]
Branch: root, Node type : root , Feature : Na_to_K ,  Parent : Na_to_K , Left Child : HIGH_Cholesterol , Right Child : HIGH_Cholesterol , NodeID :

In [17]:
df_dict['summary 1']

interested_class  base_rate  feature operator  \
decision 0 constraint 0                 2         10  HIGH_BP       <=   
           constraint 1                 2         10  Na_to_K       <=   

                         threshold  total_sample_size  \
decision 0 constraint 0     0.5000                 90   
           constraint 1    14.8285                 52   

                         interested_class_sample_size  target_lift  \
decision 0 constraint 0                            23            3   
           constraint 1                            23            3   

                         actual_lift data type  
decision 0 constraint 0         2.56    object  
           constraint 1         4.42   float64

In [18]:
df_dict['summary 2']

interested_class  base_rate             feature  \
decision 0  constraint 0                 3         10             HIGH_BP   
decision 1  constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10           NORMAL_BP   
decision 2  constraint 0                 3         10           NORMAL_BP   
            constraint 1                 3         10              LOW_BP   
decision 3  constraint 0                 3         10           NORMAL_BP   
decision 4  constraint 0                 3         10             Na_to_K   
decision 5  constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10             Na_to_K   
decision 6  constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10                 Age   
            constraint 2                 3         10             Na_to_K   
decision 7  constraint 0                 3         10           NORMAL_BP   
            constraint 1                 3         10             Na_to_K   
decision 8  constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10              LOW_BP   
decision 9  constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10                 Age   
decision 10 constraint 0                 3         10                 Age   
decision 11 constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10  NORMAL_Cholesterol   
decision 12 constraint 0                 3         10    HIGH_Cholesterol   
            constraint 1                 3         10           NORMAL_BP   
decision 13 constraint 0                 3         10                 Age   
            constraint 1                 3         10           NORMAL_BP   
decision 14 constraint 0                 3         10                 Age   
            constraint 1                 3         10              LOW_BP   
            constraint 2                 3         10             HIGH_BP   
decision 15 constraint 0                 3         10              LOW_BP   
decision 16 constraint 0                 3         10               F_Sex   
            constraint 1                 3         10              LOW_BP   
decision 17 constraint 0                 3         10              LOW_BP   
            constraint 1                 3         10             Na_to_K   
decision 18 constraint 0                 3         10             Na_to_K   
            constraint 1                 3         10                 Age   
decision 19 constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10           NORMAL_BP   
            constraint 2                 3         10                 Age   
decision 20 constraint 0                 3         10    HIGH_Cholesterol   
decision 21 constraint 0                 3         10                 Age   
            constraint 1                 3         10           NORMAL_BP   
            constraint 2                 3         10             Na_to_K   
decision 22 constraint 0                 3         10             HIGH_BP   
            constraint 1                 3         10    HIGH_Cholesterol   
decision 23 constraint 0                 3         10              LOW_BP   
            constraint 1                 3         10                 Age   
decision 24 constraint 0                 3         10              LOW_BP   
            constraint 1                 3         10  NORMAL_Cholesterol   
decision 25 constraint 0                 3         10  NORMAL_Cholesterol   
            constraint 1                 3         10           NORMAL_BP   
decision 26 constraint 0                 3         10              LOW_BP   
            constraint

In [19]:
df_dict['summary 0']

interested_class  base_rate    feature operator  \
decision 0  constraint 0                 1         10        Age        >   
            constraint 1                 1         10    HIGH_BP        >   
decision 1  constraint 0                 1         10        Age        >   
            constraint 1                 1         10    Na_to_K       <=   
            constraint 2                 1         10    HIGH_BP        >   
...                                    ...        ...        ...      ...   
decision 61 constraint 2                 1         10    HIGH_BP        >   
            constraint 3                 1         10      F_Sex        >   
decision 62 constraint 0                 1         10      M_Sex        >   
            constraint 1                 1         10     LOW_BP       <=   
            constraint 2                 1         10  NORMAL_BP       <=   

                          threshold  total_sample_size  \
decision 0  constraint 0     57.000                 11   
            constraint 1      0.500                 12   
decision 1  constraint 0     52.500                 19   
            constraint 1     14.685                 10   
            constraint 2      0.500                 12   
...                             ...                ...   
decision 61 constraint 2      0.500                 12   
            constraint 3      0.500                 44   
decision 62 constraint 0      0.500                 26   
            constraint 1      0.500                 15   
            constraint 2      0.500                 44   

                          interested_class_sample_size  target_lift  \
decision 0  constraint 0                            11            2   
            constraint 1                             4            2   
decision 1  constraint 0                            13            2   
            constraint 1                            15            2   
            constraint 2                             4            2   
...                                                ...          ...   
decision 61 constraint 2                             4            2   
            constraint 3                             6            2   
decision 62 constraint 0                            17            2   
            constraint 1                             8            2   
            constraint 2                            14            2   

                          actual_lift data type  
decision 0  constraint 0        10.00     int64  
            constraint 1         3.33    object  
decision 1  constraint 0         6.84     int64  
            constraint 1        15.00   float64  
            constraint 2         3.33    object  
...                               ...       ...  
decision 61 constraint 2         3.33    object  
            constraint 3         1.36    object  
decision 62 constraint 0         6.54    object  
            constraint 1         5.33    object  
            constraint 2         3.18    object  

[193 rows x 10 columns]